Load dataframes with acf and calculate tau

In [1]:
import pandas as pd
import numpy as np

# import from scripts
import os
current_wd = os.getcwd()
os.chdir(os.path.abspath("..\\..\\..\\isttc\\scripts"))
from calculate_tau import fit_single_exp
from cfg_global import project_folder_path
os.chdir(current_wd)

In [ ]:
results_folder = project_folder_path + 'results\\monkey\\fixation_period_1000ms\\'

### Functions

In [5]:
def calc_tau_per_unit(acf_df_, acf_cols_, start_idx_):
    acf_2d = acf_df_[acf_cols_].values
    acf_2d = acf_2d.astype(np.float64) # RuntimeWarning: overflow encountered in matmul sttc start_idx 2
    n_units = acf_2d.shape[0]
    print('Calculating taus for {}'.format(acf_2d.shape))
    
    fit_popt_a_l, fit_popt_b_l, fit_popt_c_l = [],[],[]
    fit_tau_l = []
    fit_r_squared_l = []
    fit_log_message_l = []
    
    for i in range(n_units):
        fit_popt, fit_pcov, tau, fit_r_squared, log_message = fit_single_exp(acf_2d[i,:], start_idx_)
        if  type(fit_popt) == np.ndarray:
            fit_popt_a_l.append(fit_popt[0])
            fit_popt_b_l.append(fit_popt[1])
            fit_popt_c_l.append(fit_popt[2])
        else:
            fit_popt_a_l.append(np.nan)
            fit_popt_b_l.append(np.nan)
            fit_popt_c_l.append(np.nan)
        fit_tau_l.append(tau)
        fit_r_squared_l.append(fit_r_squared)
        fit_log_message_l.append(log_message)
    
    data_df = np.vstack((fit_popt_a_l, fit_popt_b_l, fit_popt_c_l, fit_tau_l, fit_r_squared_l, fit_log_message_l)).T
    tau_df = pd.DataFrame(data_df, columns=['fit_a', 'fit_b', 'fit_c','tau', 'r_squared', 'log_message'])
    tau_df.insert(0, 'unit_id', acf_df_['unit_id'].values)

    for col in ['fit_a', 'fit_b', 'fit_c','tau', 'r_squared']:
        tau_df[col] = tau_df[col].astype(float) 
    
    return tau_df

In [ ]:
def calc_tau_per_area_mean(acf_mean_, start_idx_=1):    

    fit_popt, fit_pcov, tau, fit_r_squared, log_message = fit_single_exp(acf_mean_, start_idx_)
    
    # t = np.linspace(0, len(acf_mean_)-1, len(acf_mean_)).astype(int)
    # # print(t)
    
    # popt, pcov = curve_fit(func_single_exp, t[start_idx_:], acf_mean_[start_idx_:], maxfev=1000000000) # I used 5000, now it is like in Siegle
    # tau = 1 / popt[1]
    
    # # fit r-squared
    # y_pred = func_single_exp(t[start_idx_:], *popt)
    # fit_r_squared = r2_score(acf_mean_[start_idx_:], y_pred)

    return fit_popt, fit_pcov, tau, fit_r_squared, log_message

### Calculate tau

#### One tau per area

2 methods of doing that:
* average ACF over units and fit
* fit using all ACFs from all neurons

In [24]:
area = 'pfdl' # pfp

n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
pearsonr_trial_avg_2d = np.nanmean(pearsonr_trial_avg_df[acf_cols].values, axis=0)
fit_popt, fit_pcov, tau, fit_r_squared, log_message = fit_single_exp(pearsonr_trial_avg_2d, start_idx_=2)

acf_cols ['acf_0', 'acf_1', 'acf_2', 'acf_3', 'acf_4', 'acf_5', 'acf_6', 'acf_7', 'acf_8', 'acf_9', 'acf_10', 'acf_11', 'acf_12', 'acf_13', 'acf_14', 'acf_15', 'acf_16', 'acf_17', 'acf_18', 'acf_19']


In [25]:
pearsonr_trial_avg_2d

array([1.        , 0.19301799, 0.17380271, 0.15017917, 0.12690764,
       0.11182733, 0.0921734 , 0.08025915, 0.07329233, 0.06589106,
       0.06287036, 0.05711819, 0.05062921, 0.04433904, 0.04278728,
       0.04326979, 0.04019387, 0.0382939 , 0.04504843, 0.03197168])

In [23]:
tau*bin_size

264.33377148503706

#### One tau per unit

3 methods of doing that:
* Pearsonr trial average (as in the paper)
* STTC trial average
* STTC trial concat

todo: take ACF average over trails for a unit and fit? Or fit all ACFs with one function? (this requires calculating ACF per trial)

In [4]:
area = 'pfp' # pfp

n_lags = 20
bin_size = 50
acf_cols = ['acf_' + str(i) for i in range(n_lags)]
print('acf_cols {}'.format(acf_cols))

# pearsonr trial avg
pearsonr_trial_avg_df = pd.read_pickle(results_folder + 'binned\\' + area + '\\acf\\acf_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_pearsonr_trial_avg_df = calc_tau_per_unit(pearsonr_trial_avg_df, acf_cols, start_idx_=1)
tau_pearsonr_trial_avg_df['tau_ms'] = tau_pearsonr_trial_avg_df['tau'] * bin_size
tau_pearsonr_trial_avg_df.to_pickle(results_folder + 'binned\\' + area + '\\taus\\tau_pearsonr_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial avg
sttc_trial_avg_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_avg_df = calc_tau_per_unit(sttc_trial_avg_df, acf_cols, start_idx_=1)
tau_sttc_trial_avg_df['tau_ms'] = tau_sttc_trial_avg_df['tau'] * bin_size
tau_sttc_trial_avg_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_avg_1000ms_with_empty_50ms_20lags_df.pkl')

# sttc trial concat
sttc_trial_concat_df = pd.read_pickle(results_folder + 'non_binned\\' + area + '\\acf\\acf_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')
tau_sttc_trial_concat_df = calc_tau_per_unit(sttc_trial_concat_df, acf_cols, start_idx_=1)
tau_sttc_trial_concat_df['tau_ms'] = tau_sttc_trial_concat_df['tau'] * bin_size
tau_sttc_trial_concat_df.to_pickle(results_folder + 'non_binned\\' + area + '\\taus\\tau_sttc_trial_concat_1000ms_with_empty_50ms_20lags_df.pkl')

#### One tau per trial

3 methods of doing that:
* Pearsonr
* ACF formula
* iSTTC

### todo (metrics)

In [14]:
n_rows_with_nans = pearsonr_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(pearsonr_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_pearsonr_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_pearsonr_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))

acf n_rows_with_nans_perc 17.67955801104972, tau n_rows_with_nans_tau_perc 17.67955801104972


In [15]:
n_rows_with_nans = sttc_trial_avg_df[acf_cols].isnull().any(axis=1).sum()
n_rows_with_nans_perc = n_rows_with_nans / len(sttc_trial_avg_df) * 100
# acf_df.dropna(inplace=True)

n_rows_with_nans_tau = tau_sttc_trial_avg_df['tau_ms'].isnull().sum()
n_rows_with_nans_tau_perc = n_rows_with_nans_tau / len(tau_sttc_trial_avg_df) * 100
# tau_df.dropna(inplace=True)

print('acf n_rows_with_nans_perc {}, tau n_rows_with_nans_tau_perc {}'.format(n_rows_with_nans_perc, n_rows_with_nans_tau_perc))

acf n_rows_with_nans_perc 0.0, tau n_rows_with_nans_tau_perc 0.3683241252302026
